In [ ]:
import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
import duckdb

In [ ]:
con = duckdb.connect(database=':memory:', read_only=False)
con.execute("CREATE TABLE ist AS SELECT * FROM read_csv_auto('IST_corrected.csv')")
raw_size = con.execute("SELECT * FROM ist").fetchdf().size
print(raw_size)

In [ ]:
# Strings of comma seperated column names (variables) to be selected from the table, subcaterogized in determined time.
randomisation = "AGE, SEX, RHEP24, RASP3, RSBP, RDEF1, RDEF2, RDEF3, RDEF4, RDEF5, RDEF6, RDEF7, RDEF8, STYPE, RXASP, RXHEP"
fourteen_day_form = "DSCH, DIVH, DAP, DOAC, DGORM, DSTER, DCAA, DHAEMD, DCAREND, DTHROMB, DMAJNCH"
final_diagnosis = "DDIAGISC, DDIAGHA, DDIAGUN, DNOSTRK"
recurrent_stroke = "DRSISC, DRSH"
fourteen_day_death = "DPE, DDEAD, DDEADC"
six_months = "OCCODE"
# Join column String's
columns = ', '.join([randomisation, fourteen_day_form, final_diagnosis, recurrent_stroke, fourteen_day_death, six_months])
# Construct dataframe
df = con.execute("SELECT {} FROM ist".format(columns)).fetchdf()
# Keep track of pre-pre-processed data size
initial_size = df.size
print(initial_size)

In [ ]:
dtypes_before = df.dtypes
print(dtypes_before)

In [ ]:
# print(df.head(10))

In [ ]:
# Don't just drop NaN's, shrinks the data by a factor of 10(!)
# df = df.dropna()
# print(df.size)

In [ ]:
# TODO
# NaN's -> U in Y/N columns
# Drop NaN's
# df = df.fillna('U')

In [ ]:
# All characters to uppercase (ignore numeric columns)
# All Y/N(/U) columns are of type 'object'
df = df.apply(lambda x : str(x).upper() if type(x) == object else x)

In [ ]:
# Y -> 1
# N -> 0
# (U -> 0.5)?????
df = df.replace(('Y', 'N'), (1, 0))
# df = df.replace(('Y', 'N'), (True, False))
# df.equals(test_df)

In [ ]:
after_size = df.size
dtypes_after = df.dtypes
print(after_size)

In [ ]:
print(dtypes_after)
# RHEP24 and RASP3 become float64 (?)
# RXASP becomes int64 (?)

In [ ]:
df.to_csv('IST_corrected_processed.csv',index=False)